In [367]:
import pandas as pd 
import nltk 
import numpy as np 
nltk.download('punkt')
nltk.download('stopwords')
import re 

from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize , word_tokenize

import tensorflow 
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
import keras 
from keras.models import Sequential
from keras.layers import Dense , Input
from gensim.models import Word2Vec
from gensim.models import FastText

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aniru\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aniru\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [368]:
data = pd.read_csv(r"labeled_data.csv")

In [369]:
data = data[["class" , "tweet"]]

In [370]:
data.head()

,class,tweet
0,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [371]:
stop_words = set(stopwords.words('english'))
punctuation_pattern = r'[^\w\s]'

def preprocess(text):

    words_without_tag = [word for word in word_tokenize(text) if not word.startswith('@')]


    text_without_tag = " ".join(words_without_tag).strip()

    text_without_punctuation = re.sub(punctuation_pattern, '', text_without_tag)

    sentences = sent_tokenize(text_without_punctuation.lower())

    words = [word for sentence in sentences for word in word_tokenize(sentence) if word.lower() not in stop_words]

    return words

In [372]:
data['tweet'] = data['tweet'].apply(preprocess)

cbow_df = data 
sg_df = data 
fasttext_df = data

In [373]:
data.head()

,class,tweet
0,2,"[rt, mayasolovely, woman, nt, complain, cleani..."
1,1,"[rt, mleew17, boy, dats, cold, tyga, dwn, bad,..."
2,1,"[rt, urkindofbrand, dawg, rt, 80sbaby4life, ev..."
3,1,"[rt, c_g_anderson, viva_based, look, like, tra..."
4,1,"[rt, shenikaroberts, shit, hear, might, true, ..."


In [374]:
sentenes = [data["tweet"].loc[i] for i in range(len(data))]

In [375]:
cbow = Word2Vec(sentenes, sg = 0 , vector_size=100)
skipgram = Word2Vec(sentenes , sg = 1 , vector_size=100)
fasttext = FastText(sentences=sentenes, vector_size=100, window=5, min_count=1, workers=4, sg=1)

In [376]:
def convert_to_embedding(list_of_words , model):
    embedding = np.zeros(100)

    for i in list_of_words:
        try:
            embedding+= model.wv[i]
        except:
            continue 
    
    return embedding/len(list_of_words)


In [377]:
cbow_df['embedding'] = cbow_df.apply(lambda row: convert_to_embedding(row['tweet'], cbow), axis=1)
sg_df['embedding'] = sg_df.apply(lambda row: convert_to_embedding(row['tweet'], skipgram), axis=1)
fasttext_df['embedding'] = fasttext_df.apply(lambda row: convert_to_embedding(row['tweet'], fasttext), axis=1)



In [378]:
cbow_df['embedding'] = cbow_df['embedding'].apply(pd.to_numeric)
sg_df['embedding'] = sg_df['embedding'].apply(pd.to_numeric)
fasttext_df['embedding'] = fasttext_df['embedding'].apply(pd.to_numeric)


cbow_df['class'] = cbow_df['class'].apply(pd.to_numeric)
sg_df['class'] = sg_df['class'].apply(pd.to_numeric)
fasttext_df['class'] = fasttext_df['class'].apply(pd.to_numeric)



In [379]:
def numpy_to_tensor(array):
    return tf.convert_to_tensor(array)

In [380]:
model = Sequential()
model.add(Input(shape=(100,)))
model.add(Dense(30 , activation='relu'))
model.add(Dense(15 , activation='relu'))
model.add(Dense(3 , activation = 'softmax'))

In [381]:
def get_data_arrays(df):
    embeddings = df['embedding'].to_numpy()
    labels = df['class'].to_numpy()
    return embeddings, to_categorical(labels)

# Get training and validation data (consider splitting for robust evaluation)
train_embeddings, train_labels = get_data_arrays(cbow_df.copy())


In [382]:
x = []
for i in cbow_df['embedding'].values:
    x.append(i.astype('float32'))

In [383]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'] )

In [384]:
model.fit(np.array(x) , pd.get_dummies(cbow_df['class']).astype(int).values , epochs=100 , batch_size=32)

Epoch 1/100
775/775 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7808 - loss: 0.5781
Epoch 2/100
775/775 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8506 - loss: 0.4279
Epoch 3/100
775/775 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8593 - loss: 0.4012
Epoch 4/100
775/775 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8611 - loss: 0.3950
Epoch 5/100
775/775 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8632 - loss: 0.3813
Epoch 6/100
775/775 ━━━━━━━━━━━━━━━━━━━━ 1s 813us/step - accuracy: 0.8682 - loss: 0.3720
Epoch 7/100
775/775 ━━━━━━━━━━━━━━━━━━━━ 1s 713us/step - accuracy: 0.8658 - loss: 0.3731
Epoch 8/100
775/775 ━━━━━━━━━━━━━━━━━━━━ 1s 732us/step - accuracy: 0.8666 - loss: 0.3696
Epoch 9/100
775/775 ━━━━━━━━━━━━━━━━━━━━ 1s 824us/step - accuracy: 0.8659 - loss: 0.3698
Epoch 10/100
775/775 ━━━━━━━━━━━━━━━━━━━━ 1s 856us/step - accuracy: 0.8719 - loss: 0.3568
Epoch 11/100
775/775 ━━━━━━━━━━━━━━━━━━━━ 1s 781us/step - accuracy: 0.8672 - loss: 0.3641
Epoch 12/100
775/775 ━━━━━━━━

In [385]:
model2 = Sequential()
model2.add(Input(shape=(100,)))
model2.add(Dense(30 , activation='relu'))
model2.add(Dense(15 , activation='relu'))
model2.add(Dense(3 , activation = 'softmax'))

In [386]:
train_embeddings , train_labels = get_data_arrays(sg_df.copy())

In [387]:
x = []
for i in sg_df['embedding'].values:
    x.append(i.astype('float32'))

In [388]:
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'] )

In [389]:
model2.fit(np.array(x) , pd.get_dummies(sg_df['class']).astype(int).values , epochs=100 , batch_size=32)

Epoch 1/100


775/775 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.8093 - loss: 0.5465
Epoch 2/100
775/775 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8534 - loss: 0.4124
Epoch 3/100
775/775 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8606 - loss: 0.4008
Epoch 4/100
775/775 ━━━━━━━━━━━━━━━━━━━━ 1s 981us/step - accuracy: 0.8616 - loss: 0.3855
Epoch 5/100
775/775 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8658 - loss: 0.3788
Epoch 6/100
775/775 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8682 - loss: 0.3734
Epoch 7/100
775/775 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8668 - loss: 0.3672
Epoch 8/100
775/775 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8688 - loss: 0.3723
Epoch 9/100
775/775 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8684 - loss: 0.3649
Epoch 10/100
775/775 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8650 - loss: 0.3724
Epoch 11/100
775/775 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8703 - loss: 0.3564
Epoch 12/100
775/775 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/st

In [390]:
model3 = Sequential()
model3.add(Input(shape=(100,)))
model3.add(Dense(30 , activation='relu'))
model3.add(Dense(15 , activation='relu'))
model3.add(Dense(3 , activation = 'softmax'))

In [391]:
train_embeddings , train_labels = get_data_arrays(fasttext_df.copy())

In [392]:
x = []
for i in fasttext_df['embedding'].values:
    x.append(i.astype('float32'))

In [393]:
model3.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'] )

In [394]:
model3.fit(np.array(x) , pd.get_dummies(sg_df['class']).astype(int).values , epochs=100 , batch_size=32)

Epoch 1/100
775/775 ━━━━━━━━━━━━━━━━━━━━ 1s 817us/step - accuracy: 0.8080 - loss: 0.5463
Epoch 2/100
775/775 ━━━━━━━━━━━━━━━━━━━━ 1s 708us/step - accuracy: 0.8531 - loss: 0.4174
Epoch 3/100
775/775 ━━━━━━━━━━━━━━━━━━━━ 1s 921us/step - accuracy: 0.8539 - loss: 0.4063
Epoch 4/100
775/775 ━━━━━━━━━━━━━━━━━━━━ 1s 702us/step - accuracy: 0.8608 - loss: 0.3912
Epoch 5/100
775/775 ━━━━━━━━━━━━━━━━━━━━ 1s 763us/step - accuracy: 0.8660 - loss: 0.3763
Epoch 6/100
775/775 ━━━━━━━━━━━━━━━━━━━━ 1s 709us/step - accuracy: 0.8673 - loss: 0.3731
Epoch 7/100
775/775 ━━━━━━━━━━━━━━━━━━━━ 1s 756us/step - accuracy: 0.8658 - loss: 0.3739
Epoch 8/100
775/775 ━━━━━━━━━━━━━━━━━━━━ 1s 702us/step - accuracy: 0.8697 - loss: 0.3681
Epoch 9/100
775/775 ━━━━━━━━━━━━━━━━━━━━ 1s 705us/step - accuracy: 0.8663 - loss: 0.3720
Epoch 10/100
775/775 ━━━━━━━━━━━━━━━━━━━━ 1s 704us/step - accuracy: 0.8696 - loss: 0.3592
Epoch 11/100
775/775 ━━━━━━━━━━━━━━━━━━━━ 1s 772us/step - accuracy: 0.8709 - loss: 0.3545
Epoch 12/100
775/77